In [1]:
# standard libraries
import requests
import os
import pandas as pd
import numpy as np
import json
import time
import json

# custom functions
from api_functions.google_nearby_search import gPlaceSearch

In [2]:
# Get the neighborhood data

nyc = pd.read_csv('../data/data/nyc_geo_data_cleaned.csv', index_col=0)

In [3]:
# store response to figure out parsing
test_response = gPlaceSearch(str(nyc['latitude'][50]), str(nyc['longitude'][50]), 500, 'restaurant')

In [4]:
test_response

[{'html_attributions': [],
  'results': [{'business_status': 'OPERATIONAL',
    'geometry': {'location': {'lat': 40.59624669999999, 'lng': -73.9783571},
     'viewport': {'northeast': {'lat': 40.59759977989273,
       'lng': -73.97704967010728},
      'southwest': {'lat': 40.59490012010728, 'lng': -73.97974932989273}}},
    'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
    'icon_background_color': '#FF9E67',
    'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
    'name': 'Paraiso',
    'opening_hours': {'open_now': True},
    'photos': [{'height': 3024,
      'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102629022838675911100">Cindy Serrano</a>'],
      'photo_reference': 'Aap_uEBzE77GhdY7VDTsQqK3w2YqQvEk4qcngzHEJk3urN8_C3rYTi8dsc2tGOIHzWdbn5Kc3MC_voEsWXhTZ1D5YBOq4lZFoFa94wkM4qjxq6XRRD96dqHIeSUtXzTnRNvaOhSylP2uBIFaeiX-tNlOJ1mQEBiQGXfhGXWiIPEyDFgtiIYY',
      'width': 4032}]

In [5]:
# Let's figure out how to count the # of responses
row_restaurant_counts = []
for page in test_response:
    row_restaurant_counts.append(len(page['results']))

row_restaurant_counts

[18]

In [6]:
# next, a list of review counts

row_review_counts = []
for page in test_response:
    # print(page['results'])
    for result in page['results']:
        row_review_counts.append(result['user_ratings_total'])

print(row_review_counts)

[76, 42, 58, 3, 50, 37, 74, 96, 89, 134, 11, 58, 66, 24, 8433, 214, 82, 71]


In [7]:
# next, a list of ratings

row_ratings = []
for page in test_response:
    # print(page['results'])
    for result in page['results']:
        row_ratings.append(result['rating'])

print(row_ratings)

[3.9, 4.5, 4.3, 5, 3.6, 3.8, 4.2, 3.7, 4.2, 3.8, 5, 3.7, 3.3, 4, 4.6, 4.3, 3.9, 4]


In [8]:
# next, a list of types

row_ratings = []
for page in test_response:
    # print(page['results'])
    for result in page['results']:
        row_ratings.append(result['types'])

print(row_ratings)

[['restaurant', 'point_of_interest', 'food', 'establishment'], ['restaurant', 'point_of_interest', 'food', 'establishment'], ['meal_takeaway', 'restaurant', 'point_of_interest', 'food', 'establishment'], ['restaurant', 'point_of_interest', 'food', 'establishment'], ['meal_delivery', 'restaurant', 'point_of_interest', 'food', 'establishment'], ['restaurant', 'point_of_interest', 'food', 'establishment'], ['meal_delivery', 'meal_takeaway', 'restaurant', 'point_of_interest', 'food', 'establishment'], ['meal_delivery', 'restaurant', 'point_of_interest', 'food', 'establishment'], ['meal_delivery', 'restaurant', 'point_of_interest', 'food', 'establishment'], ['restaurant', 'point_of_interest', 'food', 'establishment'], ['restaurant', 'point_of_interest', 'food', 'establishment'], ['meal_delivery', 'meal_takeaway', 'restaurant', 'point_of_interest', 'food', 'establishment'], ['meal_delivery', 'restaurant', 'point_of_interest', 'food', 'establishment'], ['restaurant', 'point_of_interest', 'foo

In [9]:
# next, a list of price_level

price_levels = []
for page in test_response:
    # print(page['results'])
    for result in page['results']:
        try:
            price_levels.append(result['price_level'])
        except KeyError:
            price_levels.append('?')

print(price_levels)

['?', 2, '?', '?', 1, '?', 1, '?', '?', '?', '?', 2, 1, '?', 2, '?', 1, 1]


In [15]:
# Let's make the dict of results that's a bit more compact, and in a function

def googleEnrich(df=nyc, radius=1000, query='restaurant'):
    """
    Takes a dataframe and returns an enriched copy of the df

    df (dataframe): must include columns 'latitude' AND 'longitude' AND an 'id' column
    radius (int): distance in meters to query for
    query (string): query string

    returns a dictionary sorted by neighborhood id:
    query_review_counts = [] - review counts per venue
    query_ratings = [] - ratings per venue
    price_levels = [] - price level per venue, some missing values
    query_counts = [] - venue counts per page
    """
    google_dict = {}

    # go through every row
    for row in df.itertuples():
        print(row.id)
        # get data from API
        response = gPlaceSearch(str(row.latitude), str(row.longitude), radius, query)

        # parse data and assign data to temporary lists
        query_counts = [len(page['results']) for page in response]

        query_types = []
        query_review_counts = []
        query_ratings = []
        price_levels = []

        for page in response:
            for result in page['results']:
                try:
                    query_review_counts.append(result['user_ratings_total'])
                except KeyError:
                    query_review_counts.append('?')

                try:
                    query_ratings.append(result['rating'])
                except KeyError:
                    query_ratings.append('?')

                try:
                    query_types.append(result['types'])
                except KeyError:
                    query_types.append('?')

                try:
                    price_levels.append(result['price_level'])
                except KeyError:
                    price_levels.append('?')

        # store lists into dict
        google_dict[row.id] = {
            query + '_counts': query_counts,
            query + '_review_counts': query_review_counts,
            query + '_ratings': query_ratings,
            query + '_types': query_types,
            query + '_price_levels': price_levels
        }
    return google_dict

In [16]:
# call the function and store in a variable
google_dict = googleEnrich(query="", radius=1000)

nyu_2451_34572.1
nyu_2451_34572.2
nyu_2451_34572.3
nyu_2451_34572.4
nyu_2451_34572.5
nyu_2451_34572.6
nyu_2451_34572.7
nyu_2451_34572.8
nyu_2451_34572.9
nyu_2451_34572.10
nyu_2451_34572.11
nyu_2451_34572.12
nyu_2451_34572.13
nyu_2451_34572.14
nyu_2451_34572.15
nyu_2451_34572.16
nyu_2451_34572.17
nyu_2451_34572.18
nyu_2451_34572.19
nyu_2451_34572.20
nyu_2451_34572.21
nyu_2451_34572.22
nyu_2451_34572.23
nyu_2451_34572.24
nyu_2451_34572.25
nyu_2451_34572.26
nyu_2451_34572.27
nyu_2451_34572.28
nyu_2451_34572.29
nyu_2451_34572.30
nyu_2451_34572.31
nyu_2451_34572.32
nyu_2451_34572.33
nyu_2451_34572.34
nyu_2451_34572.35
nyu_2451_34572.36
nyu_2451_34572.37
nyu_2451_34572.38
nyu_2451_34572.39
nyu_2451_34572.40
nyu_2451_34572.41
nyu_2451_34572.42
nyu_2451_34572.43
nyu_2451_34572.44
nyu_2451_34572.45
nyu_2451_34572.46
nyu_2451_34572.47
nyu_2451_34572.48
nyu_2451_34572.49
nyu_2451_34572.50
nyu_2451_34572.51
nyu_2451_34572.52
nyu_2451_34572.53
nyu_2451_34572.54
nyu_2451_34572.55
nyu_2451_34572.56
n

In [17]:
with open('google_all_venues_1km.json', 'w') as fp:
    json.dump(google_dict, fp)

AttributeError: 'str' object has no attribute 'read'